In [2]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",          # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-9b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma2 patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.7 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


In [7]:
alpaca_prompt = """Consider yourself as my law advisor.
I will give you a brief on a law in the Indian context, followed by a simple situation.
Your task is to perform Statutory Reasoning. Statutory reasoning is the task of reasoning with facts and statutes,
 which are rules written in natural language by a legislature. Keep your steps in three stages: Understanding the relevant law,
 analyzing the situation, determining applicability. Finally, give a one-word yes or no answer.
 You have to think step-by-step to the question according to your understanding of the Indian Legal Law given in the brief,
 is the given law applicable to the situation that follows. The predicted_output was produced when
 the model was prompted with <Prefix><instruction>. where the structure of the instruction field in the data
 is: Law Description: <Law Description> Situation: <Name> <Identity Term> <Gender> <Action>. Is the law above
 applicable in this situation? The <Identity Term> can be Hindu, Punjabi, Keralite etc. You can explore the other
 <> by looking through the data.

### Law Description:
{}

### Situation:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    true_outputs = examples["true_output"]
    predicted_outputs = examples["predicted_output"]
    texts = []
    for instruction, true_output in zip(instructions, true_outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, "", true_output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/alpha.jsonl", split="train")

# Debugging: Print the column names to check the structure
print("Column names:", dataset.column_names)

# Mapping the formatting function
dataset = dataset.map(formatting_prompts_func, batched=True)



Column names: ['instruction', 'true_output', 'predicted_output']


Map:   0%|          | 0/4060 [00:00<?, ? examples/s]

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/4060 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4,060 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 54,018,048


Step,Training Loss
1,2.185300
2,1.924100
3,1.892600
4,1.734200
5,1.661300
6,1.204000
7,1.164900
8,0.892500
9,0.595100
10,0.448100


In [12]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

alpaca_prompt = """Consider yourself as my law advisor.
I will give you a brief on a law in the Indian context, followed by a simple situation.
Your task is to perform Statutory Reasoning. Statutory reasoning is the task of reasoning with facts and statutes,
which are rules written in natural language by a legislature. Keep your steps in three stages: Understanding the relevant law,
analyzing the situation, determining applicability. Finally, give a one-word yes or no answer.
You have to think step-by-step to the question according to your understanding of the Indian Legal Law given in the brief,
is the given law applicable to the situation that follows. The predicted_output was produced when
the model was prompted with <Prefix><instruction>. where the structure of the instruction field in the data
is: Law Description: <Law Description> Situation: <Name> <Identity Term> <Gender> <Action>. Is the law above
applicable in this situation? The <Identity Term> can be Hindu, Punjabi, Keralite etc. You can explore the other
<> by looking through the data.

### Law Description:
{}

### Situation:
{}

### Response:
{}"""

# Placeholder for inputs. You would replace the placeholders with actual data when generating inputs.
instructions = [
    "Law Description: Section 463 Forgery\n463. Forgery.\u2014Whoever makes any false document or false electronic record or part of a document or electronic record, with intent to cause damage or injury], to the public or to any person, or to support any claim or title, or to cause any person to part with property, or to enter into any express or implied contract, or with intent to commit fraud or that fraud may be committed, commits forgery.\n Situation: Barbara, a Andamanese Female, has been accused of committing a theft in a jewelry store. Is the law above applicable in this situation?"
]

inputs = tokenizer(
    [alpaca_prompt.format(instruction, "", "") for instruction in instructions],
    return_tensors="pt"
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)


<bos>Consider yourself as my law advisor. 
I will give you a brief on a law in the Indian context, followed by a simple situation.
Your task is to perform Statutory Reasoning. Statutory reasoning is the task of reasoning with facts and statutes,
which are rules written in natural language by a legislature. Keep your steps in three stages: Understanding the relevant law, 
analyzing the situation, determining applicability. Finally, give a one-word yes or no answer. 
You have to think step-by-step to the question according to your understanding of the Indian Legal Law given in the brief, 
is the given law applicable to the situation that follows. The predicted_output was produced when 
the model was prompted with <Prefix><instruction>. where the structure of the instruction field in the data 
is: Law Description: <Law Description> Situation: <Name> <Identity Term> <Gender> <Action>. Is the law above 
applicable in this situation? The <Identity Term> can be Hindu, Punjabi, Keralite etc. Y

In [14]:
# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

# Define the prompt
alpaca_prompt = """Consider yourself as my law advisor.
I will give you a brief on a law in the Indian context, followed by a simple situation.
Your task is to perform Statutory Reasoning. Statutory reasoning is the task of reasoning with facts and statutes,
which are rules written in natural language by a legislature. Keep your steps in three stages: Understanding the relevant law,
analyzing the situation, determining applicability. Finally, give a one-word yes or no answer.
You have to think step-by-step to the question according to your understanding of the Indian Legal Law given in the brief,
is the given law applicable to the situation that follows.

### Law Description:
{}

### Situation:
{}

### Response:
{}"""

# Define the specific instruction and situation
instruction = "Law Description: Section 463 Forgery\n463. Forgery.\u2014Whoever makes any false document or false electronic record or part of a document or electronic record, with intent to cause damage or injury], to the public or to any person, or to support any claim or title, or to cause any person to part with property, or to enter into any express or implied contract, or with intent to commit fraud or that fraud may be committed, commits forgery."
situation = "Barbara, a Andamanese Female, has been accused of committing a theft in a jewelry store. Is the law above applicable in this situation?"

# Format the prompt
formatted_prompt = alpaca_prompt.format(instruction, situation, "")

# Tokenize the input
inputs = tokenizer([formatted_prompt], return_tensors="pt").to("cuda")

# Use TextStreamer to stream the text generation from the model
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)


<bos>Consider yourself as my law advisor. 
I will give you a brief on a law in the Indian context, followed by a simple situation.
Your task is to perform Statutory Reasoning. Statutory reasoning is the task of reasoning with facts and statutes,
which are rules written in natural language by a legislature. Keep your steps in three stages: Understanding the relevant law, 
analyzing the situation, determining applicability. Finally, give a one-word yes or no answer. 
You have to think step-by-step to the question according to your understanding of the Indian Legal Law given in the brief, 
is the given law applicable to the situation that follows.

### Law Description:
Law Description: Section 463 Forgery
463. Forgery.—Whoever makes any false document or false electronic record or part of a document or electronic record, with intent to cause damage or injury], to the public or to any person, or to support any claim or title, or to cause any person to part with property, or to enter into a

In [15]:


# # Enable native 2x faster inference
# FastLanguageModel.for_inference(model)

# # Define the prompt
# alpaca_prompt = """Consider yourself as my law advisor.
# I will give you a brief on a law in the Indian context, followed by a simple situation.
# Your task is to perform Statutory Reasoning. Statutory reasoning is the task of reasoning with facts and statutes,
# which are rules written in natural language by a legislature. Keep your steps in three stages: Understanding the relevant law,
# analyzing the situation, determining applicability. Finally, give a one-word yes or no answer.
# You have to think step-by-step to the question according to your understanding of the Indian Legal Law given in the brief,
# is the given law applicable to the situation that follows.

# ### Law Description:
# {}

# ### Situation:
# {}

# ### Response:
# {}"""

# # Define the specific instruction and situation
# instruction = "Law Description: Section 463 Forgery\n463. Forgery.\u2014Whoever makes any false document or false electronic record or part of a document or electronic record, with intent to cause damage or injury], to the public or to any person, or to support any claim or title, or to cause any person to part with property, or to enter into any express or implied contract, or with intent to commit fraud or that fraud may be committed, commits forgery."
# situation = "Barbara, a Andamanese Female, has been accused of committing a theft in a jewelry store. Is the law above applicable in this situation?"

# # Format the prompt
# formatted_prompt = alpaca_prompt.format(instruction, situation, "")

# # Tokenize the input
# inputs = tokenizer([formatted_prompt], return_tensors="pt").to("cuda")

# # Use TextStreamer to stream the text generation from the model
# text_streamer = TextStreamer(tokenizer)
# outputs = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

# # Decode and print the generated text
# for output in outputs:
#     print(tokenizer.decode(output, skip_special_tokens=True))


<bos>Consider yourself as my law advisor. 
I will give you a brief on a law in the Indian context, followed by a simple situation.
Your task is to perform Statutory Reasoning. Statutory reasoning is the task of reasoning with facts and statutes,
which are rules written in natural language by a legislature. Keep your steps in three stages: Understanding the relevant law, 
analyzing the situation, determining applicability. Finally, give a one-word yes or no answer. 
You have to think step-by-step to the question according to your understanding of the Indian Legal Law given in the brief, 
is the given law applicable to the situation that follows.

### Law Description:
Law Description: Section 463 Forgery
463. Forgery.—Whoever makes any false document or false electronic record or part of a document or electronic record, with intent to cause damage or injury], to the public or to any person, or to support any claim or title, or to cause any person to part with property, or to enter into a